In [1]:
import re

def detect_all_eng(text):
    if not isinstance(text, str):
        return ''
    
    text = re.sub(r'[0-9]', '', text)
    
    if re.fullmatch(r"[A-Za-z\s]*", text):
        return False
    else:
        return text

In [2]:
def remove_invalid_reviews(df, column_name):
    try:
        filtered_df = df[df[column_name].apply(lambda text: isinstance(text, str) and text.strip() != '')]
        return filtered_df
    except Exception as e:
        print(f"Error: {e}")
        return df

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 예측 수행 함수
def predict(texts):
    model = AutoModelForSequenceClassification.from_pretrained("./hate_speech_model")
    tokenizer = AutoTokenizer.from_pretrained("./hate_speech_model")

    # 입력 데이터 토크나이징 및 디바이스 이동
    encodings = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
        add_special_tokens=True,
        stride=32,
    )

    # 모델 추론
    with torch.no_grad():  # 예측 시에는 Gradient 계산 비활성화
        outputs = model(
            input_ids=encodings["input_ids"],
            attention_mask=encodings["attention_mask"]
        )
    logits = outputs.logits

    probabilities = torch.softmax(logits, dim=-1).numpy()

    # 클래스와 확률 추출
    predictions = []
    for text, prob in zip(texts, probabilities):
        max_prob = max(prob)  # 가장 높은 확률
        max_label = prob.argmax()  # 가장 높은 확률의 클래스(숫자)
        predictions.append({"text": text, "label": max_label, "probability": max_prob})

    return predictions

# 비속어 데이터셋으로 필터링 하기

In [4]:
import pandas as pd

badwords = pd.read_csv('./Dataset/badwords.csv')['badwords'].dropna().tolist()

In [5]:
import re

def filtering_with_csv(text):
    if not isinstance(text, str):
        return ''
    
    badwords_pattern = re.compile(r'\b(' + '|'.join(map(re.escape, badwords)) + r')\b')
    filtered_text = badwords_pattern.sub('###', text)

    return filtered_text

In [6]:
import pandas as pd

# 입력 데이터
labeled = pd.read_csv('./output/output_overwatch_first_labeled.csv')
labeled.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1, inplace=True)

labeled['review'] = labeled['review'].apply(detect_all_eng)
labeled['review'] = labeled['review'].apply(filtering_with_csv)
labeled = remove_invalid_reviews(labeled, 'review')
labeled['hate'] = labeled['hate'].apply(lambda x: int(x) if pd.notnull(x) else 8)

labeled_texts = labeled['review'].tolist()
hate = labeled['hate'].tolist()
voted_up = labeled['voted_up'].tolist()

In [7]:
predictions = predict(labeled_texts)
print("Predictions:", predictions[:5])

Predictions: [{'text': '핵 잡아', 'label': 8, 'probability': 0.99730444}, {'text': '진짜 지금가지 해본 게임 중 가장 최악이다', 'label': 8, 'probability': 0.9960257}, {'text': '거이 준상이 수능 점수', 'label': 8, 'probability': 0.9978613}, {'text': '정신병 걸림', 'label': 8, 'probability': 0.9966826}, {'text': '진짜 개좆같네 팀원들이 싹다 ### ### 아오 ### 패버리고 싶다 ### 키리코 개쌔끼는 지가 딜런줄 알아 ### 그ㅜ래놓고 힐  ### ### 아오 로드호그 ### 킬 어시 뒤짐 ### 진짜 지금 생각해보니까 ### 빡치네 ### 보고있냐 ###', 'label': 3, 'probability': 0.98356116}]


In [8]:
diff = 0
for i, result in enumerate(predictions):
    if result['label'] != hate[i]:
        print(f"문장: {result['text']}")
        print(f"기존 클래스: {hate[i]}, 판단된 클래스: {result['label']}, 확률: {result['probability']:.2f}")
        print()
        diff += 1
        
print(f"기존과 예측이 달라진 개수: {diff}")

문장: ###
기존 클래스: 3, 판단된 클래스: 8, 확률: 1.00

문장: 그까짓 힘으로 감히 빨리 한국 영웅 더 주세요 ### 안에 안 주면 블리자드 앞에 오물풍선 투척함
기존 클래스: 3, 판단된 클래스: 8, 확률: 1.00

문장: ### ### 재미없는 게임 애들이 하는지 도 이해가 안감
기존 클래스: 3, 판단된 클래스: 8, 확률: 1.00

문장: 주노가 재미있다
기존 클래스: 3, 판단된 클래스: 8, 확률: 1.00

문장: 병신게임 빠대 탈주 페널티 없애라
기존 클래스: 8, 판단된 클래스: 3, 확률: 0.70

문장: 주먹 주먹 주먹 주먹 주먹 주먹 주먹
기존 클래스: 3, 판단된 클래스: 8, 확률: 1.00

문장: ㅈ망겜ㅋㅋ
기존 클래스: 8, 판단된 클래스: 3, 확률: 0.95

문장: 걍 모르겠고 배틀넷 로그인 자체가 진짜 뭣같음
기존 클래스: 3, 판단된 클래스: 8, 확률: 0.93

문장: 애들 패치를 하면 누구는 각성하는데 누군는 ㅂ신이 되는거냐
기존 클래스: 0, 판단된 클래스: 3, 확률: 0.97

문장: 탈출은 지능순입니다 유저 수준이 너무 떨어지고 게임도 다 망해서 유저가 없어요 항상 중국인 껴있습니다 즐기는 사람들은 다 접고 성질 더러운 진짜들만 남은 게임입니다 혹시라도 복귀 할까 싶으면 마음 접으시길 바랍니다
기존 클래스: 3, 판단된 클래스: 0, 확률: 0.73

문장: 캐릭터 디자인 BGM 타격감이 뛰어나고 미국 카툰식 화풍도 깔끔한 느낌을 준다 협동 슈터게임으로서 게임성 자체는 완벽에 가깝다고 생각한다 그에 반해 협동 게임 특유의 정치질 게이머가 많다는 점과 과도하게 비싼 부분결제 요소는 꽤나 거슬리는 편이다
기존 클래스: 3, 판단된 클래스: 8, 확률: 0.98

문장: 솜브라 패치는 진짜 말도 안된다 이러면 누가 솜브라 씀 할거면 제대로 해라 제발
기존 클래스: 3, 판단된 클래스: 8, 확률: 1.00

문장: 좆버워치
기존 클래스: 8, 판단된 클래스: 3, 확률: 0.97

문장: 유

'진짜 개좆같네 팀원들이 싹다 ### ### 아오 ### 패버리고 싶다 ### 키리코 개쌔끼는 지가 딜런줄 알아 ### 그ㅜ래놓고 힐  ### ### 아오 로드호그 ### 킬 어시 뒤짐 ### 진짜 지금 생각해보니까 ### 빡치네 ### 보고있냐 ###' 이 문장에서 '개좆같네', '개쌔끼'는 필터링이 되지 않았음을 알 수 있다.

# 정규식을 이용한 두 번째 필터링

In [9]:
import re

CURSEWORD_PATTERN = re.compile(r'[시씨씪슈쓔쉬쉽쒸쓉](?:[0-9]*|[0-9]+ *)[바발벌빠빡빨뻘파팔펄]|[섊좆좇졷좄좃좉졽썅춍봊]|[ㅈ조][0-9]*까|ㅅㅣㅂㅏㄹ?|ㅂ[0-9]*ㅅ|[ㅄᄲᇪᄺᄡᄣᄦᇠ]|[ㅅㅆᄴ][0-9]*[ㄲㅅㅆᄴㅂ]|[존좉좇][0-9 ]*나|[자보][0-9]+지|보빨|[봊봋봇봈볻봁봍] *[빨이]|[후훚훐훛훋훗훘훟훝훑][장앙]|[엠앰]창|애[미비]|애자|[가-탏탑-힣]색기|(?:[샊샛세쉐쉑쉨쉒객갞갟갯갰갴겍겎겏겤곅곆곇곗곘곜걕걖걗걧걨걬] *[끼키퀴])|새 *[기키퀴]|[병븅][0-9]*[신딱딲]|미친[가-닣닥-힣]|[믿밑]힌|[염옘][0-9]*병|[샊샛샜샠섹섺셋셌셐셱솃솄솈섁섂섓섔섘]기|[섹섺섻쎅쎆쎇쎽쎾쎿섁섂섃썍썎썏][스쓰]|[지야][0-9]*랄|니[애에]미|갈[0-9]*보[^가-힣]|[뻐뻑뻒뻙뻨][0-9]*[뀨큐킹낑)|꼬[0-9]*추|곧[0-9]*휴|[가-힣]슬아치|자[0-9]*박꼼|빨통|[사싸](?:이코|가지|[0-9]*까시)|육[0-9]*시[랄럴]|육[0-9]*실[알얼할헐]|즐[^가-힣]|찌[0-9]*(?:질이|랭이)|찐[0-9]*따|찐[0-9]*찌버거|창[녀놈]|[가-힣]{2,}충[^가-힣]|[가-힣]{2,}츙|부녀자|화냥년|환[양향]년|호[0-9]*[구모]|조[선센][징]|조센|[쪼쪽쪾](?:[발빨]이|[바빠]리)|盧|무현|찌끄[레래]기|(?:하악){2,}|하[앍앜]|[낭당랑앙항남담람암함][ ]?[가-힣]+[띠찌]|느[금급]마|文在|在寅|(?<=[^\n])[家哥]|속냐|[tT]l[qQ]kf|Wls|[ㅂ]신|[ㅅ]발|[ㅈ]밥|장애인')  # 해당 정규식 전체를 복사해서 붙여넣으세요

def filtering_with_re_ex(text):
    if not isinstance(text, str):
        return ''
    
    # 욕설을 ***로 대체
    filtered_text = CURSEWORD_PATTERN.sub('***', text)
    return filtered_text

In [10]:
sec_data = []

for i, result in enumerate(predictions):
    sec_data.append({
        "review": result['text'],
        "voted_up": voted_up[i],
        "fst_hate": hate[i],
        "sec_hate": result['label'],
        "sec_probability": result['probability']
    })
    
sec_df = pd.DataFrame(sec_data)

In [11]:
sec_df['review'] = sec_df['review'].apply(filtering_with_re_ex)
sec_df = remove_invalid_reviews(sec_df, 'review')

sec_texts = sec_df['review'].tolist()
hate = sec_df['sec_hate'].tolist()

In [12]:
predictions = predict(sec_texts)
print("Predictions:", predictions[:5])

Predictions: [{'text': '핵 잡아', 'label': 8, 'probability': 0.99730444}, {'text': '진짜 지금가지 해본 게임 중 가장 최악이다', 'label': 8, 'probability': 0.9960257}, {'text': '거이 준상이 수능 점수', 'label': 8, 'probability': 0.9978613}, {'text': '정신병 걸림', 'label': 8, 'probability': 0.9966826}, {'text': '진짜 개***같네 팀원들이 싹다 ### ### 아오 ### 패버리고 싶다 ### 키리코 개쌔끼는 지가 딜런줄 알아 ### 그ㅜ래놓고 힐  ### ### 아오 로드호그 ### 킬 어시 뒤짐 ### 진짜 지금 생각해보니까 ### 빡치네 ### 보고있냐 ###', 'label': 3, 'probability': 0.9637561}]


In [13]:
for result in predictions:
    print(f"문장: {result['text']}")
    print(f"판단된 클래스(숫자): {result['label']}, 확률: {result['probability']:.2f}")
    print()

문장: 핵 잡아
판단된 클래스(숫자): 8, 확률: 1.00

문장: 진짜 지금가지 해본 게임 중 가장 최악이다
판단된 클래스(숫자): 8, 확률: 1.00

문장: 거이 준상이 수능 점수
판단된 클래스(숫자): 8, 확률: 1.00

문장: 정신병 걸림
판단된 클래스(숫자): 8, 확률: 1.00

문장: 진짜 개***같네 팀원들이 싹다 ### ### 아오 ### 패버리고 싶다 ### 키리코 개쌔끼는 지가 딜런줄 알아 ### 그ㅜ래놓고 힐  ### ### 아오 로드호그 ### 킬 어시 뒤짐 ### 진짜 지금 생각해보니까 ### 빡치네 ### 보고있냐 ###
판단된 클래스(숫자): 3, 확률: 0.96

문장: ###
판단된 클래스(숫자): 8, 확률: 1.00

문장: 미국 사막지대 고속도로 위에 덩그러니 있는 이상한 술집에서 서빙하는 탈색머리 성병 매춘부 같은 이름을 가진 캐서디라는 카우보이가 출연하는 하프언럭키 하프럭키 콩코드임
판단된 클래스(숫자): 8, 확률: 0.83

문장: 학원 토껴가며 했던 친구 랭크에 열등감 갖던 시절의 나에게 귀빵망이 때려주고 싶은 게임
판단된 클래스(숫자): 8, 확률: 0.99

문장: 하나님이 보고있다면 욕함
판단된 클래스(숫자): 8, 확률: 0.99

문장: 무료자누 개꿀이자누
판단된 클래스(숫자): 8, 확률: 1.00

문장: 빠대 매칭이 ### 팀 킬뎃이 배 차이나게 매칭이 되냐 이딴식으로 쳐 잡을거면 ***련들아 일반전 패넢티를 쳐 주지 말던가 무력하게 지는 판 연속으롭 붙여놓고 나가면 나가냐고 ### 떠는거 보면 블자 본사 타이어로 다 터트리고 싶음
판단된 클래스(숫자): 3, 확률: 0.98

문장: 옵치 오베때부터 쭈욱 시간을 했지만 할때마다 스트레스받는 시간이 압도적으로 많음 그냥 안하는걸 추천 예전엔 재밌었지 나온이후로 점점 이상해지는거 같더니 지금은 두가지 경우밖에없음 아무리 똥꼬쑈를 해도 결국에는 지는판 팀에 트롤있는 경우 내 할거만 어느정도 하면 별 노력도 안하는데 그냥 이기는판 팀이 정상인 경

In [14]:
diff = 0
for i, result in enumerate(predictions):
    if result['label'] != hate[i]:
        print(f"문장: {result['text']}")
        print(f"기존 클래스: {hate[i]}, 판단된 클래스: {result['label']}, 확률: {result['probability']:.2f}")
        print()
        diff += 1
        
print(f"기존과 예측이 달라진 개수: {diff}")

문장: 그냥 ***게임
기존 클래스: 3, 판단된 클래스: 8, 확률: 1.00

문장: ***뒤진 겜 힐러가 개 ### 딜러보다 쎄면 누가 딜러함 롤처럼 딜은 딜만 힐은 힐만 탱은 탱만하게 해야지 ### 뭐 아나가 힐 안하고 딜쳐하고 모이라가 딜쳐하고 ***겜 할바엔 폰허브 솔져 리퍼 게이***나 보러가셈 Worst fxxing game ever Rather than playing this game youre better off watching soldier and reaper gay ### uploaded to pxxxhub
기존 클래스: 3, 판단된 클래스: 8, 확률: 0.89

문장: ******게임
기존 클래스: 3, 판단된 클래스: 8, 확률: 1.00

문장: ***들아 총이 m거리에서 피해감소 돠는게 말이 되냐
기존 클래스: 3, 판단된 클래스: 8, 확률: 0.98

문장: ***게임 빠대 탈주 페널티 없애라
기존 클래스: 3, 판단된 클래스: 8, 확률: 0.99

문장: 애들 패치를 하면 누구는 각성하는데 누군는 ***이 되는거냐
기존 클래스: 3, 판단된 클래스: 8, 확률: 1.00

문장: ***버워치
기존 클래스: 3, 판단된 클래스: 8, 확률: 1.00

문장: 개발자 ***들아 하다하다 이제 방벽 관통 밖에 없는 라붕이를 건드리냐 개***들아 내 라마트라 돌려내 할꺼면 라마 응징 대미지 자체를 배로 하던가 아님 라인과 정커퀸도 방벽 못 뚫게 막아 ### 개발진들아
기존 클래스: 3, 판단된 클래스: 8, 확률: 0.71

문장: 얘네가 뭔잘못을 했냐 신규 모드라면서 팀포 그대로 배낀 PVE를 만들겠다고 했다가 취소라도 했냐 트레일러도 팀포mvm 그대로 배끼기라도 했냐 한정스킨이라고 못쳐박다가 갑자기 상점에 돈주고 사게 쳐내기라도 했냐솔까 근데 이건 난 좋음ㅋㅋ 신캐들을 무조건 배패로 쳐풀어서 ***같게 하기라도 했냐 별 ***없는 모드는 쳐내면서 하나무라를 없애기라도 했냐 후 억까는 자제해주세요 ***이고 할만함
기존 클래스:

# 필터링된 문자를 삭제하고 다시 예측

In [15]:
import pandas as pd
import re

# '###'와 '***'를 모두 삭제하는 함수
def remove_placeholders(text):
    if not isinstance(text, str):
        return ''
    
    text = re.sub(r'###|\*\*\*', '', text)
    return text.strip()

In [16]:
trd_data = []

fst_hate = sec_df['fst_hate'].tolist()
sec_pro = sec_df['sec_probability'].tolist()


for i, result in enumerate(predictions):
    trd_data.append({
        "review": result['text'],
        "voted_up": voted_up[i],
        "fst_hate": fst_hate[i],
        "sec_hate": hate[i],
        "sec_probability": sec_pro[i],
        "trd_hate": result['label'],
        "trd_probability": result['probability'],
    })

trd_df = pd.DataFrame(trd_data)

In [17]:
trd_df['review'] = trd_df['review'].apply(remove_placeholders)
trd_df = remove_invalid_reviews(trd_df, 'review')

In [18]:
trd_df.columns

Index(['review', 'voted_up', 'fst_hate', 'sec_hate', 'sec_probability',
       'trd_hate', 'trd_probability'],
      dtype='object')

In [19]:
trd_texts = trd_df['review'].tolist()
hate = trd_df['trd_hate'].tolist()

In [20]:
trd_df.shape

(2494, 7)

In [21]:
sec_df.shape

(2524, 5)

In [22]:
predictions = predict(trd_texts)
print("Predictions:", predictions[:5])

Predictions: [{'text': '핵 잡아', 'label': 8, 'probability': 0.99730444}, {'text': '진짜 지금가지 해본 게임 중 가장 최악이다', 'label': 8, 'probability': 0.9960257}, {'text': '거이 준상이 수능 점수', 'label': 8, 'probability': 0.9978613}, {'text': '정신병 걸림', 'label': 8, 'probability': 0.9966826}, {'text': '진짜 개같네 팀원들이 싹다   아오  패버리고 싶다  키리코 개쌔끼는 지가 딜런줄 알아  그ㅜ래놓고 힐    아오 로드호그  킬 어시 뒤짐  진짜 지금 생각해보니까  빡치네  보고있냐', 'label': 3, 'probability': 0.9766675}]


In [24]:
diff = 0
for i, result in enumerate(predictions):
    if result['label'] != hate[i]:
        print(f"문장: {result['text']}")
        print(f"기존 클래스: {hate[i]}, 판단된 클래스: {result['label']}, 확률: {result['probability']:.2f}")
        print()
        diff += 1
        
print(f"기존과 예측이 달라진 개수: {diff}")

문장: 개롤 하다가 추억삼아 옵치 와봤는데 그렇게 망겜소리 듣는 옵치가 롤보다 재밌노
기존 클래스: 8, 판단된 클래스: 3, 확률: 0.91

문장: ㅈ겜 매칭돌리면 우리팀에 무조건 던지는놈이나 포꼬있음
기존 클래스: 8, 판단된 클래스: 3, 확률: 0.56

문장: 겜 밸런스 죶박고 지들끼리 우리겜 쩌는듯하며 자위하는 개발사 다른겜 살짝만 찍먹해봐도 이게임이  밸런스인지 느껴짐 죶나 노력해야 킬을 따는게 아닌 그냥 처 쏘다보면 얻어걸리는 캐릭터가 훨신 좋음 노력하는게 죄악인 게임 유저수준도 들인게 뭔  한국사람하고 한국말로 대화하는데  대화가 안통함 말귀를 처 못알아먹음 힐안되니까 바꿔달라고 딜 약하니까 바꿔주세요 이게  입터는거냐고  금기어 수준으로 알아들음 바꿔달라는게 니가 같이 못해서 뭐가 안되니까 바꿔가 아니라 다른걸로 하면 좋을거 같은데 이지 들이  대가리에 상식도 안들어찬새끼들이 겜하겠다고 하는게 꼴보기가 싫음 나가서 사람하고 대화를 이딴식으로하면 내가봤을때 나가자마자 분안에 칼침맞고 병원감 의사쌤이 뭐하다 칼침맞았냐 물어보면 이새끼들은 니가 알아서 뭐할건데 입털어 이럴새끼들임
기존 클래스: 8, 판단된 클래스: 3, 확률: 0.96

문장: 허구한 날 무료급식소 찾아와서 줄서서 배식받는 입장에 뚫린 입이라고 배급해주는 음식의 수준에 투정부리고 불평해대는 걸뱅이새끼들 마냥 온갖 천애고아새끼들이 플레이가 본인 입맛대로 안되면 현란한 채팅 뽐내며 본인 정신질환자다 자랑하는 게임ㅋㅋ 외 한남과 여왕벌 힐러 혜지듀오 패작 정신병자 핵쟁이 온갖  외눈박이새끼들을 판마다 매칭시켜주는  랜덤박스같은게임ㅋㅋ  쓰레기같은 게임ㅋㅋ오버워치탈출은 지능순ㅋㅋ
기존 클래스: 3, 판단된 클래스: 0, 확률: 0.36

문장: 핵 있어도 상관없음 걍 재밌으면 됨 h티어 높으면 자랑하는거 말고 인생에 도움되는게 뭐임 폐미니스트들이 명품산다고 돈낭비 해서 허영심 채우는거랑 뭐가 다름 앰생 여미새 폐미h 초딩시절에 이거 하던애들이 나이먹고 본게 많아서 깨달은게 가장 유력하고

In [26]:
final = []

voted_up = trd_df['voted_up'].tolist()
fst_hate = trd_df['fst_hate'].tolist()
sec_hate = trd_df['sec_hate'].tolist()
sec_pro = trd_df['sec_probability'].tolist()
trd_hate = trd_df['trd_hate'].tolist()
trd_probability = trd_df['trd_probability'].tolist()

for i, result in enumerate(predictions):
    final.append({
        "review": result['text'],
        "voted_up": voted_up[i],
        "fst_hate": fst_hate[i],
        "sec_hate": sec_hate[i],
        "sec_probability": sec_pro[i],
        "trd_hate": trd_hate[i],
        "trd_probability": trd_probability[i],
        "final_hate": result['label'],
        "final_probability": result['probability'],
    })

final_df = pd.DataFrame(final)

In [27]:
final_df.to_csv('filtered_overwatch.csv')